In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
# Model set up libraries 

import tensorflow as tf
import tensorflow_io as tfio

from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

2022-06-08 17:04:32.621935: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 17:04:32.621963: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:
import glob
from PIL import Image
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd

In [13]:
root_path = '../raw_data/AerialImageDataset/train/'
folders = ['images', 'gt']
folder_path = [f'{root_path}{folder}/*.tif' for folder in folders]
folder_path

['../raw_data/AerialImageDataset/train/images/*.tif',
 '../raw_data/AerialImageDataset/train/gt/*.tif']

# Loading Images

In [14]:
train_images, train_mask = [], []
train_images =[filename for filename in glob.glob(folder_path[0])]
train_mask = [filename for filename in glob.glob(folder_path[1])]

train_images.sort()
train_mask.sort()

In [15]:
# train_images

In [16]:
train_df = pd.DataFrame()
# train_df['file_path'] = train_images
train_df['image_path'] = train_images
train_df['mask_path'] = train_mask

In [17]:
train_df.head()

,image_path,mask_path
0,../raw_data/AerialImageDataset/train/images/au...,../raw_data/AerialImageDataset/train/gt/austin...
1,../raw_data/AerialImageDataset/train/images/au...,../raw_data/AerialImageDataset/train/gt/austin...
2,../raw_data/AerialImageDataset/train/images/au...,../raw_data/AerialImageDataset/train/gt/austin...
3,../raw_data/AerialImageDataset/train/images/au...,../raw_data/AerialImageDataset/train/gt/austin...
4,../raw_data/AerialImageDataset/train/images/au...,../raw_data/AerialImageDataset/train/gt/austin...


In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def holdout(df, train_ratio=0.8, test_to_val_ratio=0.5, include_all=False):

    img_paths = df["image_path"].values
    msk_paths = df["mask_path"].values

    df_mask = df.copy()

    df_train, df_val = train_test_split(df_mask, train_size=train_ratio)
    df_test, df_val = train_test_split(df_val, test_size=test_to_val_ratio)

    ds_train = tf.data.Dataset.from_tensor_slices(
         (df_train["image_path"].values, df_train["mask_path"].values)
    )
    ds_val = tf.data.Dataset.from_tensor_slices(
        (df_val["image_path"].values, df_val["mask_path"].values)
    )
    ds_test = tf.data.Dataset.from_tensor_slices(
        (df_test["image_path"].values, df_test["mask_path"].values)
    )

    return ds_train, ds_val, ds_test

In [19]:
ds_train, ds_val, ds_test = holdout(train_df)

2022-06-08 17:04:35.957242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-08 17:04:35.957261: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 17:04:35.957279: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hsth-ThinkPad-2nd): /proc/driver/nvidia/version does not exist
2022-06-08 17:04:35.958116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
def process_path(input_path, mask_path):
    """
    Load images from files.
    :input_path: the path to the satellite file
    :mask_path: the path to the mask file
    :return: The image and mask
    .. note:: Works with jpg images 
              Only the first channel is kept for the mask
    """
    
    IMAGE_SQ_SIZE = 224

    img = tf.io.read_file(input_path)
    img = tfio.experimental.image.decode_tiff(img)
    # Resize the image to the desired size
    input_img =  tf.image.resize(img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

    mask_img = tf.io.read_file(mask_path)
    mask_img = tfio.experimental.image.decode_tiff(mask_img)
    # Resize the image to the desired size
    mask_img =  tf.image.resize(mask_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])
   
    return input_img, mask_img

def normalize(image, mask):
    # image = tf.cast(image, tf.float32) / 255.

    return tf.math.divide(image, 255), tf.math.divide(mask, 255) 

In [21]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# ds_train = ds_train.map(process_path) \
# .map(normalize) \
# .batch(batch_size=64) \
# .prefetch(buffer_size=AUTOTUNE)

In [22]:
model_path_and_filename = '../models/first_UNET_input_shape_224x224x3.h5'
model = keras.models.load_model(model_path_and_filename)

In [ ]:
# very long, text output

In [92]:
# this model was compiled very simply:
# def compile_model(m):
#     m.compile(
#         loss='binary_crossentropy',
#         optimizer='adam' 
#     )
#     return m

In [23]:
# model.summary() # very long output

In [24]:
# model.get_config() # very long, text output

{'name': 'first_VGG16_UNET',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 224, 224, 3),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_25'},
   'name': 'input_25',
   'inbound_nodes': []},
  {'class_name': 'Conv2D',
   'config': {'name': 'block1_conv1',
    'trainable': False,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'same',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'block1_conv1',
   'inbound_nodes': [[['input_25', 0, 0, {}]]]},
  {'class_name

In [25]:
# # threw a good long error - tif related...
# history = model.fit(
# ds_train,
# epochs=1, 
# verbose=1)

In [26]:
from lwb_smr.CustomDataLoader import CustomDataLoader
import os

In [27]:
x_images = os.listdir('../raw_data/AerialImageDataset/train/images/')
x_path = '../raw_data/AerialImageDataset/train/images/'
y_masks = os.listdir('../raw_data/AerialImageDataset/train/gt/')
y_path = '../raw_data/AerialImageDataset/train/gt/'
input_image_size = (250, 250)
batch_size = 16

In [28]:
x_images[0], y_masks[0]

('kitsap27.tif', 'kitsap27.tif')

In [29]:
x_path, y_path

('../raw_data/AerialImageDataset/train/images/',
 '../raw_data/AerialImageDataset/train/gt/')

In [30]:
customdata = CustomDataLoader(x_images, x_path, y_masks, y_path, input_image_size, batch_size)

In [31]:
x, y = customdata[0]

2022-06-08 17:04:37.553601: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown fie

In [32]:
x.shape, y.shape

(TensorShape([16, 224, 224, 3]), TensorShape([16, 224, 224, 1]))

In [33]:
# # this now runs fine, but throws lots of :
### TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
# history = model.fit(
# customdata,
# epochs=1, 
# verbose=1)

In [34]:
# /home/hsth/code/hughharford/lwb_smr/raw_data/AerialImageDataset/train/images